In [1]:
import pandas as pd
import joblib

In [2]:
df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

# df_train = df_train.drop(columns=['FG%','3P%','FT%','BLK'])
# df_test = df_test.drop(columns=['FG%','3P%','FT%','BLK'])

In [3]:
def getAbsolute(df, name):
    df = df.abs()
    joblib.dump(df, "../data/processed/abs_"+name)
    return df

In [4]:
df_train = getAbsolute(df_train, "train")
df_test = getAbsolute(df_test, "test")

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df_train.iloc[:,2:-1]
X


,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,22.6,2.0,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6
1,75,21.8,10.5,4.2,7.9,55.1,0.3,1.0,34.9,2.4,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4
2,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,34.3,0.4,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6
3,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,23.7,0.9,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9
4,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,13.7,0.2,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,32,9.2,1.8,0.7,1.8,40.3,0.1,0.2,23.1,0.4,0.6,65.7,0.3,1.8,1.9,0.5,0.3,0.2,0.4
7996,54,6.0,1.8,0.7,1.4,48.7,0.1,0.1,3.1,0.2,0.4,70.1,1.0,1.1,2.0,0.1,0.0,0.3,0.3
7997,85,28.2,10.7,4.0,9.0,45.1,0.2,0.6,23.6,2.8,3.9,69.7,1.0,2.1,3.1,3.4,1.2,0.2,1.8
7998,39,7.7,2.5,1.0,2.3,40.1,0.3,0.5,13.3,0.6,0.7,74.3,0.4,0.6,0.9,0.2,0.3,0.3,0.5


In [7]:
y = df_train.iloc[:,-1:]
y

,TARGET_5Yrs
0,1
1,1
2,1
3,1
4,1
...,...
7995,1
7996,1
7997,1
7998,1


In [8]:
df_train.TARGET_5Yrs.value_counts()

1    6669
0    1331
Name: TARGET_5Yrs, dtype: int64

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=8)
y_train.value_counts()

TARGET_5Yrs
1              5326
0              1074
dtype: int64

In [10]:
y_val.value_counts()

TARGET_5Yrs
1              1343
0               257
dtype: int64

In [11]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler().fit(X_train)
X_train = sc.transform(X_train)
X_val = sc.transform(X_val)

In [12]:
X_test = df_test.iloc[:,2:]
X_test
X_test = sc.transform(X_test)

In [13]:
X_test_id = df_test.iloc[:,1:2]
X_test_id

,Id
0,0
1,1
2,2
3,3
4,4
...,...
3794,3794
3795,3795
3796,3796
3797,3797


In [14]:
joblib.dump(X_train, "../data/processed/X_train")
joblib.dump(y_train, "../data/processed/y_train")
joblib.dump(X_val, "../data/processed/X_val")
joblib.dump(y_val, "../data/processed/y_val")
joblib.dump(X_test, "../data/processed/X_test")
joblib.dump(X_test_id, "../data/processed/X_test_id")

['../data/processed/X_test_id']